In [40]:
from typing import Annotated, Literal

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from langchain_core.messages import HumanMessage
import os
import keyring

# API KEY
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_mac')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

# Tavily tool
tavily_tool = TavilySearchResults(max_results=5)

# Code execution tool
python_repl_tool = PythonREPLTool()


In [41]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [HumanMessage(content=result["messages"][-1].content, name=name)]
    }
    
members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perfrom a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# Out team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members


In [42]:
class routeResponse(BaseModel):
    next: Literal[tuple(options)]

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should be next?"
            " or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model='gpt-4o-mini')

In [44]:
def supervisor_agent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    return supervisor_chain.invoke(state)

In [45]:
import functools
import operator
from typing import Sequence, TypedDict
from langchain_core.messages import BaseMessage
from display_graph import display_graph

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

# The agent state is in the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str
    
research_agent = create_react_agent(llm, tools=[tavily_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Reseacher")

# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_react_agent(llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

In [46]:
# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("Supervisor", supervisor_agent)

for member in members:
    # We want out worker to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "Supervisor")

# The supervisor populates the "next" fields in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map)

# Finally add entrypoint
workflow.add_edge(START, "Supervisor")

graph = workflow.compile()

# Visualize the graph
display_graph(graph)

/Users/woojin/Documents/github/learning/llm/langgraph/langgraph_blueprint/ch13/graphs/graph_55291.png


In [47]:
for s in graph.stream(
    {"messages": [
        HumanMessage(content="Research recent advances in AI and summarize.")
    ]}
):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content="Recent advances in artificial intelligence (AI) throughout 2023 have made significant contributions across various sectors, marking it as a pivotal year in the field. Here are the highlights:\n\n1. **Generative AI Improvements**: The year has seen remarkable progress in generative AI models, particularly with technologies like ChatGPT and models for image generation (e.g., Stable Diffusion and MidJourney). These advancements have enhanced the capabilities of AI in content creation and automation.\n\n2. **Natural Language Processing (NLP)**: Advances have been made in NLP, with models becoming more sophisticated in understanding and generating human-like text, enabling better interaction between users and AI systems.\n\n3. **Integration into Daily Life**: AI technologies have moved from laboratory settings into everyday applications, significantly with the introduction of AI-driven tools that 

In [48]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Write a brief pdf researh report on pikas.")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'Supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='It appears that I cannot directly create a PDF at this moment. However, I can guide you on how to create one using the report content. Here\'s a step-by-step process to create a PDF on your own:\n\n### Instructions to Create a PDF\n\n1. **Copy the Report Content**:\n   Copy the entire text of the research report provided above.\n\n2. **Open a Word Processor**:\n   Open a program like Microsoft Word, Google Docs, or any text editor that allows you to save documents as PDF.\n\n3. **Paste the Content**:\n   Paste the copied report into the document.\n\n4. **Format the Document**:\n   - You can adjust the headings, font size, and spacing as you prefer to improve readability.\n   - Add a title at the top, such as "Research Report on Pikas".\n\n5. **Save as PDF**:\n   - If you’re using Microsoft Word, go to \'File\' > \'Save As\' and choose PDF from the format options.\n   - If you are using Google

Python REPL can execute arbitrary code. Use with caution.


{'Coder': {'messages': [HumanMessage(content="I can't create a PDF directly at the moment, but I can provide you with the content for a research report on pikas. You can then create a PDF using the steps I provided earlier. Here's the content:\n\n---\n\n# Research Report on Pikas\n\n## Introduction\nPikas are small, mountain-dwelling mammals that belong to the family Ochotonidae. They are closely related to rabbits and hares, sharing a common ancestor. Pikas are found in North America and Asia, primarily in rocky, alpine environments.\n\n## Habitat and Distribution\nPikas inhabit rocky slopes and talus fields, where they find shelter in crevices. They are typically found at elevations above 2,500 meters (8,200 feet) in North America and can adapt to various climates, but prefer cooler temperatures.\n\n## Behavior and Diet\nPikas are herbivores, feeding on a diet of grasses, herbs, and other vegetation. They are known for their characteristic behavior of storing food for the winter mont